# 強化学習と線型計画法（有限ホライゾン）

参考文献

* [Exploration-Exploitation in Constrained MDPs](https://arxiv.org/abs/2003.02189)の2.3章

強化学習が扱う最適方策の導出（プランニング問題）は線形計画問題としても定式化できます。
有限ホライゾンの場合でも、無限ホライゾン([RL_as_LP.ipynb](RL_as_LP.ipynb))と似たような形式で線形計画問題に落とし込むことができます。

**表記**（[RL_utils.ipynb](RL_utils.ipynb)参照）

MDPを次で定義します。

1. 有限状態集合: $S=\{1, \dots, |S|\}$
2. 有限行動集合: $A=\{1, \dots, |A|\}$
3. $h$ステップ目の遷移確率行列: $P_h\in \mathbb{R}^{SA\times S}$
4. $h$ステップ目の報酬行列: $r_h\in \mathbb{R}^{S\times A}$
5. ホライゾン: $H$
6. 初期状態: $\mu \in \mathbb{R}^{S}$

また、次の占有率を導入しておきます。　
* 占有率：$d_h^\pi(s, a ; p):=\mathbb{E}\left[\mathbb{1}\left\{s_h=s, a_h=a\right\} \mid s_1=s_1, p, \pi\right]=\operatorname{Pr}\left\{s_h=s, a_h=a \mid s_1=s_1, p, \pi\right\}$
* 価値関数：$V_1^\pi\left(s_1\right)=\sum_{h, s, a} d_h^\pi(s, a) r_h(s, a):=r^T d^\pi(p)$


占有率は任意の$h \in[H] \backslash\{1\}$について次を満たします。

$$
\begin{aligned}
\sum_a d_h^\pi(s, a) & =\sum_{s^{\prime}, a^{\prime}} p_{h-1}\left(s \mid s^{\prime}, a^{\prime}\right) d_{h-1}^\pi\left(s^{\prime}, a^{\prime}\right) & & \forall s \in \mathcal{S} \\
d_h^\pi(s, a) & \geq 0 & & \forall s, a
\end{aligned}
$$

$h=1$については
$$d_1^\pi(s, a)=\pi_1(a \mid s) \cdot \mu(s) \quad \forall s, a$$

です。

これを使うと、有限ホライゾンの最適方策は

$$
\begin{aligned}
& \max_d \sum_{s, a, h} d_h(s, a) r_h(s, a) &&\\
\text { s.t. } &\sum_a d_h(s, a) =\sum_{s^{\prime}, a^{\prime}} p_{h-1}\left(s \mid s^{\prime}, a^{\prime}\right) d_{h-1}\left(s^{\prime}, a^{\prime}\right) & & \forall h \in[H] \backslash\{1\} \\
& \sum_a d_1(s, a) =\mu(s) & & \forall s \in \mathcal{S}\\
& d_h(s, a) \geq 0 & &\forall(s, a, h) \in \mathcal{S} \times \mathcal{A} \times[H]
\end{aligned}
$$

を解き、

$$
\pi_h^d(a \mid s)=\frac{d_h(s, a)}{\sum_b d_h(s, b)}, \quad \forall(s, a, h) \in \mathcal{S} \times \mathcal{A} \times[H]
$$

とすれば求まります。

In [14]:
import jax
import jax.numpy as jnp
import numpy as np
from typing import NamedTuple, Optional
from jax.random import PRNGKey

key = PRNGKey(0)

S = 10  # 状態集合のサイズ
A = 3  # 行動集合のサイズ
S_set = jnp.arange(S)  # 状態集合
A_set = jnp.arange(A)  # 行動集合
H = 30  # ホライゾン

# 報酬行列を適当に作ります
key, _ = jax.random.split(key)
rew = jax.random.uniform(key=key, shape=(H, S, A))
assert rew.shape == (H, S, A)


# 遷移確率行列を適当に作ります
key, _ = jax.random.split(key)
P = jax.random.uniform(key=key, shape=(H, S*A, S))
P = P / jnp.sum(P, axis=-1, keepdims=True)  # 正規化して確率にします
P = P.reshape(H, S, A, S)
np.testing.assert_allclose(P.sum(axis=-1), 1, atol=1e-6)  # ちゃんと確率行列になっているか確認します


# 初期状態分布を適当に作ります
key, _ = jax.random.split(key)
init_dist = jax.random.uniform(key, shape=(S,))
init_dist = init_dist / jnp.sum(init_dist)
np.testing.assert_allclose(init_dist.sum(axis=-1), 1, atol=1e-6)  # ちゃんと確率行列になっているか確認します


# 状態集合, 行動集合, 割引率, 報酬行列, 遷移確率行列が準備できたのでMDPのクラスを作ります

class MDP(NamedTuple):
    S_set: jnp.array  # 状態集合
    A_set: jnp.array  # 行動集合
    H: int  # ホライゾン
    rew: jnp.array  # 報酬行列
    P: jnp.array  # 遷移確率行列
    init_dist: jnp.array  # 初期分布
    optimal_Q: Optional[jnp.ndarray] = None  # 最適Q値

    @property
    def S(self) -> int:  # 状態空間のサイズ
        return len(self.S_set)

    @property
    def A(self) -> int:  # 行動空間のサイズ
        return len(self.A_set)


mdp = MDP(S_set, A_set, H, rew, P, init_dist)

print("状態数：", mdp.S)
print("行動数：", mdp.A)
print("ホライゾン：", mdp.H)

状態数： 10
行動数： 3
ホライゾン： 30


### （準備）動的計画法

**表記**

* ステップ$h$の方策行列（$\Pi_h^\pi \in \mathbb{R}^{S\times SA}$）：$\langle \pi_h, q\rangle$を行列で書きたいときに便利。
    * $\Pi_h^\pi(s,(s, a))=\pi_h(a \mid s)$ 
    * $\Pi_h^\pi q_h^\pi = \langle \pi, q_h^\pi \rangle = v_h^\pi$が成立。
* ステップ$h$の遷移確率行列１（$P_h^\pi \in \mathbb{R}^{SA\times SA}$）: 次の状態についての方策の情報を追加したやつ。
    * $P_h^\pi = P_h \Pi_h^\pi$
    * Q値を使ったベルマン期待作用素とかで便利。$q_h^\pi = r_h + P_h^\pi q^\pi$が成立。
* ステップ$h$の遷移確率行列２（$\bar{P}_h^\pi \in \mathbb{R}^{S\times S}$）: 方策$\pi$のもとでの状態遷移の行列。
    * $\bar{P}_h^\pi = \Pi_h^\pi P_h$
    * V値を使ったベルマン期待作用素とかで便利。$v_h^\pi = \Pi^\pi r_h + \gamma \bar{P}_h^\pi v^\pi$。
* ステップ$h$の訪問頻度（$d^\pi_{h,\mu} \in \mathbb{R}^{SA}$）：S, Aについての累積訪問頻度
    * ${d}^\pi_{h,\mu} (s, a) = \pi(a|s) \sum_{s_0} \mu(s_0) \sum_{t=0}^h \mathrm{Pr}\left(S_t=s|S_0=s_0, M(\pi)\right)$


**実装した関数**

* ``compute_greedy_policy``: Q関数 ($H\times S \times A \to \mathcal{R}$) の貪欲方策を返します
* ``compute_optimal_Q``: MDPの最適Q関数 $q_* : H\times S \times A \to \mathcal{R}$ を返します。
* ``compute_policy_Q``: 方策 $\pi$ のQ関数 $q_\pi : H\times S \times A \to \mathcal{R}$ を返します。
* ``compute_policy_matrix``: 方策$\pi$の行列${\Pi}^{\pi} : H \times S \times SA$を返します。
* ``compute_policy_visit``: 方策 $\pi$ の割引訪問頻度${d}^\pi_{\mu} : {H\times S \times A}$ を返します。

In [15]:
from functools import partial
import jax
import chex


@jax.jit
def compute_greedy_policy(Q: jnp.ndarray):
    """Q関数の貪欲方策を返します

    Args:
        Q (jnp.ndarray): (HxSxA)の行列

    Returns:
        greedy_policy (jnp.ndarray): (HxSxA)の行列
    """
    greedy_policy = jnp.zeros_like(Q)
    H, S, A = Q.shape
    
    def body_fn(i, greedy_policy):
        greedy_policy = greedy_policy.at[i, jnp.arange(S), Q[i].argmax(axis=-1)].set(1)
        return greedy_policy

    greedy_policy = jax.lax.fori_loop(0, H, body_fn, greedy_policy)
    chex.assert_shape(greedy_policy, (H, S, A))
    return greedy_policy


@partial(jax.jit, static_argnames=("H", "S", "A"))
def _compute_optimal_Q(mdp: MDP, H: int, S: int, A: int):
    """ベルマン最適作用素をホライゾン回走らせて最適価値関数を動的計画法で計算します。
    Args:
        mdp (MDP)

    Returns:
        optimal_Q (jnp.ndarray): (HxSxA)の行列
    """

    def backup(i, optimal_Q):
        h = H - i - 1
        max_Q = optimal_Q[h+1].max(axis=1)
        next_v = mdp.P[h] @ max_Q
        chex.assert_shape(next_v, (S, A))
        optimal_Q = optimal_Q.at[h].set(mdp.rew[h] + next_v)
        return optimal_Q
    
    optimal_Q = jnp.zeros((H+1, S, A))
    optimal_Q = jax.lax.fori_loop(0, mdp.H, backup, optimal_Q)
    return optimal_Q[:-1]

compute_optimal_Q = lambda mdp: _compute_optimal_Q(mdp, mdp.H, mdp.S, mdp.A)


@jax.jit
def compute_policy_Q(mdp: MDP, policy: jnp.ndarray):
    """ベルマン期待作用素をホライゾン回走らせて価値関数を動的計画法で計算します。
    Args:
        mdp (MDP)
        policy (np.ndarray): (HxSxA)の行列

    Returns:
        optimal_Q (jnp.ndarray): (HxSxA)の行列
    """
    H, S, A = policy.shape

    def backup(i, policy_Q):
        h = H - i - 1
        max_Q = (policy[h+1] * policy_Q[h+1]).sum(axis=1)
        next_v = mdp.P[h] @ max_Q
        chex.assert_shape(next_v, (S, A))
        policy_Q = policy_Q.at[h].set(mdp.rew[h] + next_v)
        return policy_Q
    
    policy_Q = jnp.zeros((H+1, S, A))
    policy_Q = jax.lax.fori_loop(0, mdp.H, backup, policy_Q)
    return policy_Q[:-1]


@jax.jit
def compute_policy_matrix(policy: jnp.ndarray):
    """
    上で定義した方策行列を計算します。方策についての内積が取りたいときに便利です。
    Args:
        policy (jnp.ndarray): (HxSxA)の行列

    Returns:
        policy_matrix (jnp.ndarray): (HxSxSA)の行列
    """
    H, S, A = policy.shape
    PI = policy.reshape(H, 1, S, A)
    PI = jnp.tile(PI, (1, S, 1, 1))
    eyes = jnp.tile(jnp.eye(S).reshape(1, S, S, 1), (H, 1, 1, 1))
    PI = (eyes * PI).reshape(H, S, S*A)
    return PI


@jax.jit
def compute_policy_visit(mdp: MDP, policy: jnp.ndarray, init_dist: jnp.ndarray):
    """MDPと方策について、訪問頻度を動的計画法で計算します。
    Args:
        mdp (MDP)
        policy (jnp.ndarray): (HxSxA)の行列
        init_dist (jnp.ndarray): (S) 初期状態の分布

    Returns:
        visit (jnp.ndarray): (HxSxA)のベクトル
    """
    H, S, A = policy.shape
    Pi = compute_policy_matrix(policy)
    P = mdp.P.reshape(H, S*A, S)

    def body_fn(h, visit):
        next_visit = visit[h] @ P[h] @ Pi[h+1]
        visit = visit.at[h+1].set(next_visit)
        return visit
    
    visit = jnp.zeros((H+1, S*A))
    visit = visit.at[0].set((init_dist @ Pi[0]))
    visit = jax.lax.fori_loop(0, mdp.H, body_fn, visit)
    visit = visit[:-1].reshape(H, S, A)
    return visit


# 動的計画法による最適価値関数
optimal_Q_DP = compute_optimal_Q(mdp)
optimal_V_DP = optimal_Q_DP.max(axis=-1)
optimal_policy = compute_greedy_policy(optimal_Q_DP)
optimal_policy_Q_DP = compute_policy_Q(mdp, optimal_policy)
mdp = mdp._replace(optimal_Q=optimal_Q_DP)
print("最適価値関数と最適方策の価値関数の差", jnp.abs(optimal_Q_DP - optimal_policy_Q_DP).max())

# 訪問頻度によるリターンの計算
policy_visit = compute_policy_visit(mdp, optimal_policy, mdp.init_dist)
np.testing.assert_allclose(policy_visit.sum(axis=(1, 2)), 1.0, atol=1e-6)
np.testing.assert_allclose(policy_visit[0].sum(axis=-1), mdp.init_dist, atol=1e-6)
for h in range(H):
    return_by_visit = (policy_visit * mdp.rew)[h:].sum()
    return_by_DP = (optimal_Q_DP[h] * policy_visit[h]).sum()
    print(f"{h}ステップ目の訪問頻度によるリターンと動的計画法によるリターンの差", np.abs(return_by_visit - return_by_DP))

最適価値関数と最適方策の価値関数の差 0.0
0ステップ目の訪問頻度によるリターンと動的計画法によるリターンの差 1.9073486e-06
1ステップ目の訪問頻度によるリターンと動的計画法によるリターンの差 1.9073486e-06
2ステップ目の訪問頻度によるリターンと動的計画法によるリターンの差 1.9073486e-06
3ステップ目の訪問頻度によるリターンと動的計画法によるリターンの差 0.0
4ステップ目の訪問頻度によるリターンと動的計画法によるリターンの差 1.9073486e-06
5ステップ目の訪問頻度によるリターンと動的計画法によるリターンの差 1.9073486e-06
6ステップ目の訪問頻度によるリターンと動的計画法によるリターンの差 5.722046e-06
7ステップ目の訪問頻度によるリターンと動的計画法によるリターンの差 9.536743e-06
8ステップ目の訪問頻度によるリターンと動的計画法によるリターンの差 9.536743e-06
9ステップ目の訪問頻度によるリターンと動的計画法によるリターンの差 3.8146973e-06
10ステップ目の訪問頻度によるリターンと動的計画法によるリターンの差 3.8146973e-06
11ステップ目の訪問頻度によるリターンと動的計画法によるリターンの差 9.536743e-07
12ステップ目の訪問頻度によるリターンと動的計画法によるリターンの差 9.536743e-07
13ステップ目の訪問頻度によるリターンと動的計画法によるリターンの差 9.536743e-07
14ステップ目の訪問頻度によるリターンと動的計画法によるリターンの差 0.0
15ステップ目の訪問頻度によるリターンと動的計画法によるリターンの差 0.0
16ステップ目の訪問頻度によるリターンと動的計画法によるリターンの差 9.536743e-07
17ステップ目の訪問頻度によるリターンと動的計画法によるリターンの差 9.536743e-07
18ステップ目の訪問頻度によるリターンと動的計画法によるリターンの差 2.861023e-06
19ステップ目の訪問頻度によるリターンと動的計画法によるリターンの差 1.9073486e-06
20ステップ目の訪問頻度によるリターンと動的計画法によるリターンの差 1.4305115e-

In [16]:
import pulp
from itertools import product
prob = pulp.LpProblem(name="CMDP", sense=pulp.LpMaximize)
hsa_indices = [(h, s, a) for h, s, a in product(range(H), range(S), range(A))]
sa_indices = [(s, a) for s, a in product(range(S), range(A))]
d = pulp.LpVariable.dicts("d", hsa_indices, lowBound=0, cat="Continuous")

# 目的関数
prob += pulp.lpSum([d[hsa] * mdp.rew[hsa[0], hsa[1], hsa[2]] for hsa in hsa_indices])

# 初期状態についての制約
for s in range(S):
    d_0sa = [d[(0, s, a)] for a in range(A)]
    prob += pulp.lpSum(d_0sa) == mdp.init_dist[s].item()

# 各ステップについての制約
for h in range(1, H):
    for ns in range(S):
        d_hns = pulp.lpSum([d[(h, ns, na)] for na in range(A)])
        d_phns = pulp.lpSum([d[(h-1, sa[0], sa[1])] * mdp.P[h-1, sa[0], sa[1], ns] for sa in sa_indices])
        prob += d_hns == d_phns


sol = prob.solve()
d_arr = jnp.array([pulp.value(d[h, s, a]) for (h, s, a) in hsa_indices])
d_arr = d_arr.reshape(H, S, A)

np.testing.assert_allclose(d_arr.sum(axis=(1, 2)), 1.0, atol=1e-4)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/toshinori/shumi-note/.venv/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/d7e19f5f14da4d2684e66562025be7db-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/d7e19f5f14da4d2684e66562025be7db-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 305 COLUMNS
At line 10806 RHS
At line 11107 BOUNDS
At line 11108 ENDATA
Problem MODEL has 300 rows, 900 columns and 9600 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 190 (-110) rows, 550 (-350) columns and 5950 (-3650) elements
0  Obj -0 Primal inf 1.9339191 (10) Dual inf 1046.5361 (550)
0  Obj -0 Primal inf 1.9339191 (10) Dual inf 1.3796436e+12 (550)
31  Obj -0 Primal inf 1.9339191 (10) Dual inf 5.542992e+12 (445)
62  Obj -0 Primal inf 1.9339191 (10) Dual inf 7.0748679e+12 (370)
93  Obj -0 Primal inf 1.9339191 (10) Dual inf 8.326

In [17]:
policy = d_arr / d_arr.sum(axis=-1, keepdims=True)
Q_LP = compute_policy_Q(mdp, policy)

print("最適価値関数とLPによる価値関数の差", jnp.abs(optimal_Q_DP - Q_LP).max())

最適価値関数とLPによる価値関数の差 0.0


PULPを使ってもいいですが，かなり遅いです．なんとか通常の行列形式に書き換えてみましょう．

元の問題は

$$
\begin{aligned}
& \max_d \sum_{s, a, h} d_h(s, a) r_h(s, a) &&\\
\text { s.t. } &\sum_a d_h(s, a) - \sum_{s^{\prime}, a^{\prime}} p_{h-1}\left(s \mid s^{\prime}, a^{\prime}\right) d_{h-1}\left(s^{\prime}, a^{\prime}\right) = 0& & \forall h \in[H] \backslash\{1\} \\
& \sum_a d_1(s, a) =\mu(s) & & \forall s \in \mathcal{S}\\
& d_h(s, a) \geq 0 & &\forall(s, a, h) \in \mathcal{S} \times \mathcal{A} \times[H]
\end{aligned}
$$

でした．この制約の部分を

$$
B d = b
$$

の形に直します．

In [71]:
from itertools import product
d = d_arr
d = d.reshape(H * S * A)

B = np.zeros((H, S, A, H, S, A))

# 初期状態についての制約
for s, a in product(range(S), range(A)):
    B[0, s, a, 0, s] = 1


# 遷移についての制約
for h, s, a in product(range(1, H), range(S), range(A)):
    B[h, s, a, h, s] = 1  # sum_a d(h, s, a) を実現します
    B[h, s, a, h-1] = -mdp.P[h-1, :, :, s]  # sum_a d(h, s, a) を実現します


B = B.reshape((H*S*A, H*S*A))
mu = np.repeat(mdp.init_dist[:, None], A, axis=1).reshape(-1)
b = np.hstack((mu, np.zeros((H-1)*S*A)))

np.testing.assert_almost_equal(B @ d, b)

これで行列形式での制約ができました．これを使って，次の問題をscipyで解きます．
$$
\begin{aligned}
& \max d^T r \;\; \text { s.t. }  B d = b \; \text{ and }\; d \geq 0
\end{aligned}
$$



In [81]:
from scipy.optimize import linprog

r = - mdp.rew.reshape(-1)
lin_res = linprog(r, A_eq=B, b_eq=b, bounds=(0, None))

d_arr_matrix = lin_res.x.reshape(H, S, A)
policy = d_arr_matrix / d_arr_matrix.sum(axis=-1, keepdims=True)
Q_LP_matrix = compute_policy_Q(mdp, policy)

print("最適価値関数と行列形式のLPによる価値関数の差", jnp.abs(optimal_Q_DP - Q_LP_matrix).max())

最適価値関数とLPによる価値関数の差 0.0
